# preparation

**# 医院病床使用情况可视化大屏**

这是一个用于实时监控香港医院病床使用情况的可视化大屏系统。

**## 功能特点**

- **占用率展示**：显示各医院及科室的病床使用率
- **空闲病床数据**：实时展示空闲病床数量及分布情况
- **病床分布情况**：通过热力图展示不同医院和科室的病床使用情况
- **信息概览**：总病床数、已用病床数、空闲病床数和整体使用率的实时数据

**## 技术栈**

- **后端**：Python + Flask
- **前端**：HTML + CSS + JavaScript
- **数据可视化**：ECharts
- **数据处理**：Pandas


**## 快速开始**

1. 安装依赖：

```bash
pip install flask pandas openpyxl matplotlib seaborn
```

2. 运行应用：

```bash
python app.py
```

3. 访问系统：

在浏览器中打开 `http://127.0.0.1:5000`

**## 文件结构**

- `app.py` - Flask应用主程序
- `hospital_bed_usage_data.xlsx` - 病床使用数据源
- `templates/index.html` - 可视化大屏前端界面
- `view_excel_data.py` - 数据分析脚本(辅助工具)

**## 可视化图表说明**

- **总体概况**：显示数据概览和科室空闲病床分布饼图
- **各医院病床使用率**：横向条形图展示各医院的病床使用率
- **主要科室病床使用率**：柱状图展示主要科室的病床使用率
- **医院-科室病床使用率热力图**：用热力图展示各医院各科室的病床使用情况 

In [ ]:
# # !pip install flask pandas openpyxl matplotlib seaborn
# OR
# !pip install -r requirements.txt


# Appliaction

# 学习/练习顺序（在本 notebook 操作）
按顺序跑下面几个单元格，边跑边看日志/数据，就能理解流程，不用把 .py 复制进来。

In [1]:
# Step 0: 安装依赖（首次/缺包时运行）
# 更新了 requirements.txt，已包含 openpyxl
# %pip install -r requirements.txt

In [2]:
# !python 1_precompute_data.py
# !python 2_view_excel_data.py
# !python 3_app.py

In [3]:
# Step 1: 快速查看原始 Excel 列和前几行
import pandas as pd

excel_path = 'hospital_bed_usage_data.xlsx'
df = pd.read_excel(excel_path)
print('列名:', list(df.columns))
df.head()

列名: ['hospital_id', 'hospital_name', 'hospital_district', 'department_id', 'department_name', 'ward_id', 'ward_name', 'total_beds', 'occupied_beds', 'available_beds', 'occupancy_rate', 'timestamp', 'special_status']


,hospital_id,hospital_name,hospital_district,department_id,department_name,ward_id,ward_name,total_beds,occupied_beds,available_beds,occupancy_rate,timestamp,special_status
0,HH001,玛丽医院,港岛,DEPT01,内科,HH001_DEPT01_W1,内科普通病房1号,23,23,0,99.00,2023-12-01,满床
1,HH001,玛丽医院,港岛,DEPT01,内科,HH001_DEPT01_W2,内科普通病房2号,20,20,0,99.00,2023-12-01,满床
2,HH001,玛丽医院,港岛,DEPT01,内科,HH001_DEPT01_W3,内科普通病房3号,28,28,0,99.00,2023-12-01,满床
3,HH001,玛丽医院,港岛,DEPT02,外科,HH001_DEPT02_W1,外科特需病房1号,6,6,0,94.69,2023-12-01,正常
4,HH001,玛丽医院,港岛,DEPT03,儿科,HH001_DEPT03_W1,儿科普通病房1号,37,35,2,94.81,2023-12-01,正常


In [4]:
# Step 2: 预计算并写入 data_cache/
%run 1_precompute_data.py

2026-01-07 21:23:30,768 - __main__ - INFO - 开始预计算数据...
2026-01-07 21:23:30,781 - __main__ - INFO - 读取Excel文件: d:\Job_hunt\2025Courses_LLM\3-Cursor编程-从入门到精通\Case2_bed_usage\hospital_bed_usage_data.xlsx
2026-01-07 21:23:36,544 - __main__ - INFO - Excel文件读取完成，共 59773 行数据
2026-01-07 21:23:36,544 - __main__ - INFO - 计算医院使用率数据...
2026-01-07 21:23:36,557 - __main__ - INFO - 计算科室使用率数据...
2026-01-07 21:23:36,569 - __main__ - INFO - 计算概览数据...
2026-01-07 21:23:36,578 - __main__ - INFO - 计算热力图数据...
2026-01-07 21:23:36,599 - __main__ - INFO - 缓存数据已保存到 d:\Job_hunt\2025Courses_LLM\3-Cursor编程-从入门到精通\Case2_bed_usage\data_cache\data_cache.pkl
2026-01-07 21:23:36,614 - __main__ - INFO - 元数据已保存到 d:\Job_hunt\2025Courses_LLM\3-Cursor编程-从入门到精通\Case2_bed_usage\data_cache\metadata.json
2026-01-07 21:23:36,616 - __main__ - INFO - 数据预计算完成，耗时: 5.85 秒


数据预计算完成，请运行 app.py 查看结果


In [5]:
# Step 3: 查看缓存结构
import pickle, json

cache = pickle.load(open('data_cache/data_cache.pkl', 'rb'))
print('缓存键:', list(cache.keys()))
print('summary_data 示例:', json.dumps(cache.get('summary_data', {}), ensure_ascii=False, indent=2))

缓存键: ['hospital_usage', 'department_usage', 'summary_data', 'heatmap_data', 'timestamp', 'excel_last_modified', 'excel_md5']
summary_data 示例: {
  "total_beds": 1443095,
  "occupied_beds": 1192292,
  "available_beds": 209855,
  "occupancy_rate": 82.62,
  "top_departments": {
    "names": [
      "眼科",
      "皮肤科",
      "耳鼻喉科",
      "泌尿科",
      "妇产科"
    ],
    "values": [
      27236,
      27179,
      22546,
      18316,
      17958
    ]
  }
}


In [6]:
# Step 4: 生成探索型图表（输出在 charts/）
%run 2_view_excel_data.py

=== 数据字段名称 ===
1. hospital_id
2. hospital_name
3. hospital_district
4. department_id
5. department_name
6. ward_id
7. ward_name
8. total_beds
9. occupied_beds
10. available_beds
11. occupancy_rate
12. timestamp
13. special_status

=== 前20行数据 ===
   hospital_id hospital_name hospital_district department_id department_name  \
0        HH001          玛丽医院                港岛        DEPT01              内科   
1        HH001          玛丽医院                港岛        DEPT01              内科   
2        HH001          玛丽医院                港岛        DEPT01              内科   
3        HH001          玛丽医院                港岛        DEPT02              外科   
4        HH001          玛丽医院                港岛        DEPT03              儿科   
5        HH001          玛丽医院                港岛        DEPT04             妇产科   
6        HH001          玛丽医院                港岛        DEPT04             妇产科   
7        HH001          玛丽医院                港岛        DEPT05              骨科   
8        HH001          玛丽医院      

D:\Job_hunt\2025Courses_LLM\3-Cursor编程-从入门到精通\Case2_bed_usage\2_view_excel_data.py:91: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x='病床使用率', y=hospital_col, data=hospital_usage_sorted, palette='viridis')


已生成图表保存在 d:\Job_hunt\2025Courses_LLM\3-Cursor编程-从入门到精通\Case2_bed_usage\charts 目录中


In [ ]:
# Step 5: 启动 Flask（会常驻，占用这个单元格）
# 推荐在外部终端运行：python 3_app.py --port 5000
# 如需在 notebook 中临时跑，执行后自动打开浏览器访问 http://127.0.0.1:5000
# 中断该单元格即可停止
import webbrowser, sys, subprocess, time

# 启动 Flask
proc = subprocess.Popen([sys.executable, "3_app.py", "--port", "5000"])

# 等待服务启动片刻，然后自动打开浏览器
time.sleep(2)
webbrowser.open("http://127.0.0.1:5000/")

# 阻塞当前单元格，直到你手动中断（Kernel -> Interrupt）
proc.wait()